In [1]:
# Step 1: Import all necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
import os

print("Libraries imported successfully!")

# %%
# Step 2: Define Parameters and Fetch Data
STOCK_TICKER = 'AAPL'
TRAIN_START_DATE = '2015-01-01'
TRAINING_DATA_RATIO = 0.8

# Fetch data using yfinance
df = yf.download(STOCK_TICKER, start=TRAIN_START_DATE)

# ✅ Fix: handle MultiIndex columns (Price, Ticker)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

# Use only 'Close' price
data = df[['Close']]
dataset = data.values  # Convert DataFrame to numpy array

# Calculate number of rows to train the model
training_data_len = int(np.ceil(len(dataset) * TRAINING_DATA_RATIO))

print(f"Data fetched for {STOCK_TICKER}.")
print(f"Total data points: {len(dataset)}")
print(f"Training data points: {training_data_len}")
print(df.head())

# %%
# Step 3: Scale the Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

print("Data scaled successfully.")

# %%
# Step 4: Create the Training Dataset
SEQUENCE_LENGTH = 60
train_data = scaled_data[0:int(training_data_len), :]

x_train = []
y_train = []

for i in range(SEQUENCE_LENGTH, len(train_data)):
    x_train.append(train_data[i-SEQUENCE_LENGTH:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape for LSTM [samples, timesteps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print(f"Training data prepared. Shape of x_train: {x_train.shape}")

# %%
# Step 5: Build the LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# %%
# Step 6: Train the Model
model.fit(x_train, y_train, batch_size=32, epochs=10)
print("\nModel training complete.")

# %%
# Step 7: Save the Trained Model and Scaler
MODELS_DIR = 'D:\Infosys_springboard\models'
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

model.save(f'{MODELS_DIR}/lstm_model.h5')
joblib.dump(scaler, f'{MODELS_DIR}/scaler.gz')

print(f"Model saved to {MODELS_DIR}/lstm_model.h5")
print(f"Scaler saved to {MODELS_DIR}/scaler.gz")


<>:85: SyntaxWarning: invalid escape sequence '\I'
<>:85: SyntaxWarning: invalid escape sequence '\I'
C:\Users\saial\AppData\Local\Temp\ipykernel_18104\2692098090.py:85: SyntaxWarning: invalid escape sequence '\I'
  MODELS_DIR = 'D:\Infosys_springboard\models'


Libraries imported successfully!


C:\Users\saial\AppData\Local\Temp\ipykernel_18104\2692098090.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(STOCK_TICKER, start=TRAIN_START_DATE)


[*********************100%***********************]  1 of 1 completed

Data fetched for AAPL.
Total data points: 2713
Training data points: 2171
                Close       High        Low       Open     Volume
Date                                                             
2015-01-02  24.261047  24.729270  23.821672  24.718174  212818400
2015-01-05  23.577578  24.110154  23.391177  24.030267  257142000
2015-01-06  23.579802  23.839432  23.218093  23.641935  263188400
2015-01-07  23.910439  24.010296  23.677436  23.788389  160423600
2015-01-08  24.829132  24.886828  24.121249  24.238861  237458000
Data scaled successfully.
Training data prepared. Shape of x_train: (2111, 60, 1)


...

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 6.8798e-04

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 6.8811e-04



Model training complete.


Model saved to D:\Infosys_springboard\models/lstm_model.h5


Scaler saved to D:\Infosys_springboard\models/scaler.gz
